In [1]:
import pandas as pd 
%matplotlib notebook

In [ ]:
from sklearn.base import BaseEstimator 
from sklearn.base import ClassifierMixin 
from sklearn.preprocessing import LabelEncoder 
from sklearn.externals import six 
from sklearn.base import clone 
from sklearn.pipeline import _name_estimators 
import numpy as np 

In [8]:
from sklearn import datasets 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder 

In [9]:
iris = datasets.load_iris()
X,y = iris.data[50:,[1,2]] ,iris.target[50:] 
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state=1,stratify=y)

In [11]:
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline

In [15]:
import numpy as np 
clf1 = LogisticRegression(penalty='l2',C=0.001,random_state=1)
clf2 = DecisionTreeClassifier(max_depth=1,criterion='entropy',random_state=0)
clf3 = KNeighborsClassifier(n_neighbors=1,p=2,metric='minkowski')


In [16]:
pipe1 = Pipeline([['sc',StandardScaler()],['clf',clf1]]) 
pipe3= Pipeline([['sc',StandardScaler()],['clf',clf3]])

clf_labels = ['Logistic regression','Decision Tree','KNN']
print('10 fold cross validation:\n')  


10 fold cross validation:



In [25]:
for clf , label in zip([pipe1,clf2,pipe3],clf_labels): 
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]" % (scores.mean(),scores.std(),label))

ROC AUC: 0.87 (+/- 0.17) [Logistic regression]
ROC AUC: 0.89 (+/- 0.16) [Decision Tree]
ROC AUC: 0.88 (+/- 0.15) [KNN]


In [32]:
import sys 
import importlib
sys.path.append('/home/krish.mahajan/Documents/PythonMachineLearning/MLbook') 
from algorithm import majority_vote_classifier
importlib.reload( majority_vote_classifier ) 

<module 'algorithm.majority_vote_classifier' from '/home/krish.mahajan/Documents/PythonMachineLearning/MLbook/algorithm/majority_vote_classifier.py'>

In [33]:
mv_clf = majority_vote_classifier.MajorityVoteClassifier(classifiers =[pipe1,clf2,pipe3]) 
clf_labels += ['Majority Voting']
all_clf = [pipe1,clf3,pipe3,mv_clf] 

    

In [34]:
for clf,label in zip(all_clf,clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]" % (scores.mean(),scores.std(),label))

ROC AUC: 0.87 (+/- 0.17) [Logistic regression]
ROC AUC: 0.86 (+/- 0.14) [Decision Tree]
ROC AUC: 0.88 (+/- 0.15) [KNN]
ROC AUC: 0.94 (+/- 0.13) [Majority Voting]


In [38]:
mv_clf.get_params()

{'decisiontreeclassifier': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=0,
             splitter='best'),
 'decisiontreeclassifier__class_weight': None,
 'decisiontreeclassifier__criterion': 'entropy',
 'decisiontreeclassifier__max_depth': 1,
 'decisiontreeclassifier__max_features': None,
 'decisiontreeclassifier__max_leaf_nodes': None,
 'decisiontreeclassifier__min_impurity_decrease': 0.0,
 'decisiontreeclassifier__min_impurity_split': None,
 'decisiontreeclassifier__min_samples_leaf': 1,
 'decisiontreeclassifier__min_samples_split': 2,
 'decisiontreeclassifier__min_weight_fraction_leaf': 0.0,
 'decisiontreeclassifier__presort': False,
 'decisiontreeclassifier__random_state': 0,
 'decisiontreeclassifier__splitter': 

In [39]:
from sklearn.model_selection import GridSearchCV 

In [41]:
params = {'decisiontreeclassifier__max_depth':[1,2],
         'pipeline-1__clf__C':[0.001,0.1,100.0]} 

grid = GridSearchCV(estimator=mv_clf ,param_grid=params,cv=10,scoring='roc_auc')
grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=MajorityVoteClassifier(classifiers=[Pipeline(memory=None,
     steps=[['sc', StandardScaler(copy=True, with_mean=True, with_std=True)], ['clf', LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', ra...ski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')]])],
            vote='classlabel', weights=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'pipeline-1__clf__C': [0.001, 0.1, 100.0], 'decisiontreeclassifier__max_depth': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [45]:
for params, mean_score, scores in grid.grid_scores_: 
    print("%0.3f+/-%0.2f %r" % (mean_score,scores.std()/2 ,params))

0.933+/-0.07 {'pipeline-1__clf__C': 0.001, 'decisiontreeclassifier__max_depth': 1}
0.947+/-0.07 {'pipeline-1__clf__C': 0.1, 'decisiontreeclassifier__max_depth': 1}
0.973+/-0.03 {'pipeline-1__clf__C': 100.0, 'decisiontreeclassifier__max_depth': 1}
0.947+/-0.07 {'pipeline-1__clf__C': 0.001, 'decisiontreeclassifier__max_depth': 2}
0.947+/-0.07 {'pipeline-1__clf__C': 0.1, 'decisiontreeclassifier__max_depth': 2}
0.973+/-0.03 {'pipeline-1__clf__C': 100.0, 'decisiontreeclassifier__max_depth': 2}


/home/krish.mahajan/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [46]:
print('Best parameters: %s' % grid.best_params_)

Best parameters: {'pipeline-1__clf__C': 100.0, 'decisiontreeclassifier__max_depth': 1}


In [47]:
print('Accuracy: %.2f' % grid.best_score_)

Accuracy: 0.97


## Applying bagging to classify samples in wine dataset

In [2]:
import pandas as pd 


In [3]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

In [6]:
#drop 1 class 
df_wine= df_wine[df_wine['Class label'] !=1]

In [7]:
y = df_wine['Class label'].values 
X = df_wine[['Alcohol','OD280/OD315 of diluted wines']].values

In [9]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
le = LabelEncoder() 
y = le.fit_transform(y) 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1,stratify=y)

In [14]:
from sklearn.ensemble import BaggingClassifier  
from sklearn.tree import DecisionTreeClassifier 
tree = DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=None) 
bag = BaggingClassifier(base_estimator=tree,n_estimators=500,max_samples=1.0,max_features=1.0,bootstrap=True,bootstrap_features=False,
                       n_jobs=1,random_state=1)

In [16]:
from sklearn.metrics import accuracy_score
tree = tree.fit(X_train,y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
tree_train = accuracy_score(y_train,y_train_pred)
tree_test = accuracy_score(y_test,y_test_pred)

print('Decision tree train/test accuracies %.3f/%.3f' %(tree_train,tree_test))

Decision tree train/test accuracies 1.000/0.833


In [18]:
bag = bag.fit(X_train,y_train)
y_train_pred = bag.predict(X_train)
y_test_pred = bag.predict(X_test)
bag_train = accuracy_score(y_train,y_train_pred)
bag_test = accuracy_score(y_test,y_test_pred) 

print('Decision tree train/test accuracies %.3f/%.3f' %(bag_train,bag_test))

Decision tree train/test accuracies 1.000/0.917


# Applying AdaBoost using scikit-learn

In [23]:
from sklearn.ensemble import AdaBoostClassifier 
tree = DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=1) 
ada = AdaBoostClassifier(base_estimator = tree, n_estimators =500,learning_rate =0.1,random_state=1) 
tree = tree.fit(X_train,y_train)
ada = ada.fit(X_train,y_train)

In [21]:
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
tree_train = accuracy_score(y_train,y_train_pred)
tree_test = accuracy_score(y_test,y_test_pred) 

print('Decision tree train/test accuracies %.3f/%.3f' %(tree_train,tree_test))

Decision tree train/test accuracies 0.916/0.875


In [24]:
y_train_pred = ada.predict(X_train)
y_test_pred = ada.predict(X_test)
ada_train = accuracy_score(y_train,y_train_pred)
ada_test = accuracy_score(y_test,y_test_pred)  


print('Adaboost train/test accuracies %.3f/%.3f' %(ada_train,ada_test))

Adaboost train/test accuracies 1.000/0.917
